In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from implementations import *
from Data_cleaning import *
from helpers import *

In [2]:
x_train, x_test, y_train, train_ids, test_ids = load_csv_data("data\dataset\dataset")

In [4]:
#Turn y into 0s and 1s
y_tr_ = (y_train + 1) / 2
y_tr = y_tr_[:240000]
y_te = y_tr_[240000:300000]
print(y_tr.shape, y_te.shape, y_tr, y_te)

NameError: name 'y_train' is not defined

In [5]:

def remove_categorical_features(X, threshold=10):
    """
    Removes categorical (low-cardinality) features from a numeric dataset.
    A feature is dropped if it has fewer than `threshold` unique values.
    """
    X_np = np.asarray(X)
    n_samples, n_features = X_np.shape
    keep_mask = np.array([
        np.unique(X_np[:, j]).size >= threshold for j in range(n_features)
    ])
    X_num = X_np[:, keep_mask]
    return X_num, keep_mask
def anova_f_test(X, y):
    """
    Compute ANOVA F-statistic for each feature using NumPy only.
    
    Parameters
    ----------
    X : np.ndarray, shape (n_samples, n_features)
        Feature matrix.
    y : np.ndarray, shape (n_samples,)
        Class labels.
    
    Returns
    -------
    F_values : np.ndarray, shape (n_features,)
        F-statistics for each feature.
    """
    n_samples, n_features = X.shape
    classes = np.unique(y)
    k = len(classes)
    
    overall_means = np.nanmean(X, axis=0)  # handle NaNs safely
    
    # Initialize sums of squares
    ssb = np.zeros(n_features)
    ssw = np.zeros(n_features)
    
    for c in classes:
        X_c = X[y == c]
        n_c = X_c.shape[0]
        mean_c = np.nanmean(X_c, axis=0)
        
        ssb += n_c * (mean_c - overall_means) ** 2
        ssw += np.nansum((X_c - mean_c) ** 2, axis=0)
    
    # Avoid division by zero
    ssw = np.where(ssw == 0, np.nan, ssw)
    
    F = (ssb / (k - 1)) / (ssw / (n_samples - k))
    
    return F


In [40]:
# --- CLEANING ---
"""
# 0. Remove categorical (low-cardinality) features
x_num, cat_mask = remove_categorical_features(x_train, threshold=10)
print(x_num.shape)
"""
# 1. Remove features with too many NaNs
x_clean, keep_mask = remove_nan_features(x_train)
x_clean_tr = x_clean[:240000]
x_clean_te = x_clean[240000:300000]

# 2. Impute remaining missing values
# Compute feature means from the training data (ignore NaNs)
train_means = np.nanmean(x_clean_tr, axis=0)
train_stds = np.nanstd(x_clean_tr, axis = 0)
# Replace NaNs in training data with training means
inds_tr = np.where(np.isnan(x_clean_tr))
x_clean_tr[inds_tr] = np.take(train_means, inds_tr[1])

# Replace NaNs in test data with training means
inds_te = np.where(np.isnan(x_clean_te))
x_clean_te[inds_te] = np.take(train_means, inds_te[1])
"""
# Compute F-scores using only the training data
F_scores = anova_f_test(x_clean_tr, y_tr)


# Select top 30 features
top_k = 
top_features_idx = np.argsort(F_scores)[-top_k:][::-1]

# Apply the same feature selection to all sets
x_anova_tr = x_clean_tr[:, top_features_idx]
x_anova_te = x_clean_te[:, top_features_idx]
"""
# --- STANDARDIZATION ---
x_tr_st = (x_clean_tr - train_means) / train_stds
x_te_st = (x_clean_te - train_means) / train_stds


print(x_tr_st.shape, y_tr.shape)

(240000, 144) (240000,)


In [41]:
x_tr_ba, y_tr_ba = balance_data(x_tr_st, y_tr)
print(x_tr_ba.shape, y_tr_ba.shape)


(42262, 144) (42262,)


In [42]:
loss, w = reg_logistic_regression(
    y_tr_ba, x_tr_ba, max_iter=10000, gamma=0.5, lambda_=1e-3, threshold=1e-8
)
print(loss, w)

Iteration     0, loss = 0.693147
Iteration   100, loss = 0.478041
Iteration   200, loss = 0.477310
Iteration   300, loss = 0.477114
Iteration   400, loss = 0.477021
Iteration   500, loss = 0.476967
Iteration   600, loss = 0.476933
Iteration   700, loss = 0.476910
Iteration   800, loss = 0.476893
Iteration   900, loss = 0.476881
Iteration  1000, loss = 0.476872
Iteration  1100, loss = 0.476864
Iteration  1200, loss = 0.476858
Iteration  1300, loss = 0.476854
Iteration  1400, loss = 0.476850
Iteration  1500, loss = 0.476846
Iteration  1600, loss = 0.476844
Iteration  1700, loss = 0.476841
Iteration  1800, loss = 0.476839
Iteration  1900, loss = 0.476838
Iteration  2000, loss = 0.476836
Iteration  2100, loss = 0.476835
Iteration  2200, loss = 0.476834
Converged at iteration 2224
0.4768334366723658 [-9.44980758e-01  8.49136315e-03  5.70460452e-02 -2.24961552e-02
 -2.24670391e-02 -3.13413543e-04 -3.71663777e-02 -2.41743373e-02
 -1.54129311e-02 -1.54129311e-02  5.36113267e-01 -7.64795173e-02

In [5]:
def predict_labels_logistic(x, w, threshold=0.75, return_original_labels=True):

    # add bias term
    tx = np.c_[np.ones(x.shape[0]), x]
    
    # compute probabilities
    probs = sigmoid(tx @ w)
    
    # threshold
    preds = (probs >= threshold).astype(int)
    
    # convert to {-1, 1} if desired
    if return_original_labels:
        preds = 2 * preds - 1
    
    return preds

In [6]:
y_pred = predict_labels_logistic(x_te_st, w, return_original_labels=False)
n_errors = np.sum(np.abs(y_te - y_pred))
accuracy = np.mean(y_te == y_pred)

print("Misclassified samples:", n_errors)
print(f"Accuracy: {accuracy:.3f}")
print(np.sum(y_pred), np.sum(y_te))

NameError: name 'x_te_st' is not defined

In [76]:
def f1_score(y_true, y_pred):
    """Compute F1 score between true and predicted labels."""
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    if tp + fp == 0 or tp + fn == 0:
        return 0.0
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)

In [77]:
print(f1_score(y_te, y_pred))

0.3829346392917687
